In [13]:
from afinetune import *

In [2]:
def load_data():
    data = h5py.File('Dataset_Specific_labelled.h5', 'r')  
    X = data['jet'][...]
    y = data['Y'][...]
    y = np.squeeze(y)
    X = rearrange(X, 'b h w c -> b c h w')
    yt = np.zeros((y.shape[0],2))
    yt[:,0] = 1 - y
    yt[:,1] = y
    y = yt
    _, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42)    
    testset = TensorDataset(torch.tensor(X_test,dtype=torch.float32), torch.tensor(y_test,dtype=torch.float32))
    testloader = DataLoader(testset, batch_size=512, shuffle=True)
    return testloader

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()
model = FineTuneModel()
model.load_state_dict(torch.load('first/model_finetuned.pth'))
model.to(device)

testloader = load_data()

In [20]:
def test_model(dataloader, model, lossfn, device):  
    model.eval()
    correct = 0
    total = 0
    total_loss = 0
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device).float(), labels.to(device).float()
            outputs = model(inputs)
            loss = lossfn(outputs, labels)
            total += labels.size(0)
            outputs = outputs.argmax(dim=1)
            labels = labels.argmax(dim=1)
            correct += (outputs == labels).sum().item()
            total_loss += loss.item()
    return correct / total , total_loss

In [21]:
acc, _ = test_model(testloader, model, criterion, device)
print("Accuracy: ", acc*100, "%")

Accuracy:  78.45 %
